In [234]:
import pandas as pd
import re
import plotly.graph_objects as go

# Funkcja do usunięcia litery na końcu wartości
def remove_last_character(value):
    pattern = r'\D$'  # Wzorzec do dopasowania ostatniego niecyfrowego znaku
    return re.sub(pattern, '', value)

# Wczytanie danych z pliku CSV
data = pd.read_csv('men_100m.csv')

# Konwersja kolumny 'Event Date' na typ daty
data['Event Date'] = pd.to_datetime(data['Event Date'], format='%d.%m.%Y')

# Usunięcie litery na końcu wartości w kolumnie 'Time'
data['Time'] = data['Time'].apply(remove_last_character)

# Konwersja kolumny 'Time' na liczbę zmiennoprzecinkową
data['Time'] = data['Time'].str.replace(',', '.').astype(float)

# Dodanie nowej kolumny 'Year' na podstawie kolumny 'Event Date'
data['Year'] = data['Event Date'].dt.year

# Grupowanie danych według roku i wybór najlepszego wyniku dla każdego roku
best_results = data.groupby('Year')['Time'].min()

# Tworzenie wykresu za pomocą Plotly
fig = go.Figure()

# Tworzenie słownika przypisującego kolor dla każdego zawodnika
colors = {
    'Usain Bolt': 'green',
    'Carl Lewis': 'red',
    # Dodaj inne zawodniki i ich kolory tutaj
}

# Dodawanie punktów dla najlepszych wyników w każdym roku
for year, time in best_results.items():
    athlete_data = data.loc[(data['Year'] == year) & (data['Time'] == time)]
    athlete = athlete_data['Athlete'].values[0]
    country = athlete_data['Country'].values[0]
    color = colors.get(athlete, 'gray')
    label = f"Zawodnik: {athlete}<br>Rok: {int(year)}<br>Czas: {time} s<br>Kraj: {country}"
    fig.add_trace(go.Scatter(
        x=[year],
        y=[time],
        mode='markers',
        name=athlete,
        text=[label],
        hovertemplate='%{text}<extra></extra>',
        marker=dict(color=color, size=15),  # Zwiększenie rozmiaru kulek
    ))

# Dodawanie prostokąta 1
fig.add_shape(
    type="rect",
    x0=1981,
    y0=9.5,
    x1=1991,
    y1=10.5,
    fillcolor="rgba(255, 0, 0, 0.2)",
    line=dict(color='rgba(255, 0, 0, 0.2)'),
    opacity=0.5,
    layer="below",
    xref='x',
    yref='y'
)

# Dodawanie prostokąta 2
fig.add_shape(
    type="rect",
    x0=2008,
    y0=9.5,
    x1=2013,
    y1=10.5,
    fillcolor="rgba(0, 255, 0, 0.2)",
    line=dict(color='rgba(0, 255, 0, 0.2)'),
    opacity=0.5,
    layer="below",
    xref='x',
    yref='y'
)

# Konfiguracja układu wykresu
fig.update_layout(
    xaxis=dict(title='Rok'),
    yaxis=dict(title='Najlepszy czas (s)'),
    title='Najlepszy wynik w biegu na 100 m',
    showlegend=True,
)

# Ustawienie skalowania osi Y
fig.update_yaxes(range=[9.5, 10.2])

# Zapisywanie wykresu do pliku HTML
fig.write_html('best_of_every_season.html')





In [239]:
import plotly.graph_objects as go
import pandas as pd

data = {
    'Country': ['USA', 'JAM', 'GBR', 'TTO', 'CAN', 'NGR', 'RSA', 'FRA', 'SKN', 'JPN'],
    'Count': [1555, 696, 236, 187, 148, 146, 121, 109, 80, 78],
    'Picture': ['US.png', 'JM.png', 'GB.png', 'TT.png', 'CA.png', 'NG.png', 'ZA.png', 'FR.png', 'KN.png', 'JP.png']
}

# Wczytanie danych do DataFrame
df = pd.DataFrame(data)

# Tworzenie figury
fig = go.Figure()

# Dodawanie markerów z obrazkami
for i, row in df.iterrows():
    fig.add_layout_image(
        dict(
            source=f"country_bubbles/{row['Picture']}",
            xref="paper",
            yref="paper",
            x=i / (len(df) - 1),
            y=0.5,
            sizex=0.2,
            sizey=0.2,
            xanchor="center",
            yanchor="middle",
            opacity=0.7,
            layer="above"
        )
    )

# Aktualizowanie osi x
fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False)

# Aktualizowanie osi y
fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False)

# Aktualizowanie układu wykresu
fig.update_layout(
    width=800,
    height=400,
    plot_bgcolor='white',
    showlegend=False
)



# Zapisywanie wykresu do pliku HTML
fig.write_html('country_bubbles.html')
